In [ ]:
import stripe
import pandas as pd
import numpy as np
import psycopg2
from time import sleep
from config import api_key
from config import host, database, user, password
from queries import *
import pandas.io.sql as sqlio


In [ ]:
def get_df():
    df = pd.DataFrame()
    stripe.api_key = api_key
                                                        
    #first query to get response id
    first_response = stripe.Subscription.list(status='all')
    #creating dataframe with responses
    df = df.append(pd.json_normalize(first_response['data']))
    #defining next page for loops
    start_after = first_response['data'][-1]['id']
    print('Getting Results')

    while True:    
        try:
            #appending to the dataframe
            response = stripe.Subscription.list(starting_after=start_after, status='all')
            subscription_df = pd.json_normalize(response['data'])
            df = df.append(subscription_df, ignore_index=True)
            #giving the key for the next iteration
            start_after = response['data'][-1]['id']
            #ask george about rate limit issues
            sleep(.1)
        except:
            print('Stripe Request went wrong')
        
        
        if response["has_more"] == False:
            break
    return df


In [ ]:
df = get_df()
df

In [ ]:
def cleaning_df(df):
    df = df[['id', 'billing_cycle_anchor', 'cancel_at', 'cancel_at_period_end', 'canceled_at', 'created', 'current_period_end', 'current_period_start', 'customer', 'ended_at', 'quantity', 'start_date', 'status']]
    df['billing_cycle_anchor'] = pd.to_datetime(df['billing_cycle_anchor'], unit='s')
    df['canceled_at'] = pd.to_datetime(df['canceled_at'], unit='s')
    df['cancel_at'] = pd.to_datetime(df['cancel_at'], unit='s')
    df['created'] = pd.to_datetime(df['created'], unit='s')
    df['current_period_end'] = pd.to_datetime(df['current_period_end'], unit='s')
    df['current_period_start'] = pd.to_datetime(df['current_period_start'], unit='s')
    df['ended_at'] = pd.to_datetime(df['ended_at'], unit='s')
    df['start_date'] = pd.to_datetime(df['start_date'], unit='s')
    df = df.replace({np.NaN: None})
    return df


In [ ]:
df = cleaning_df(df)
df

In [ ]:
def connect(host, database, user, password):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(host=host, database=database, user=user, password=password)
        conn.autocommit = True
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    return conn
conn = connect(host, database, user, password)

In [ ]:
def setup(conn, table):
    cur = conn.cursor()
    cur.execute(table)
    

In [ ]:
setup(conn, create_subscription_table)

In [ ]:
df.columns

In [ ]:
def insert_values(conn, df):
    
    cur = conn.cursor()
    
    for value in df.values:
        subscription_id, billing_cycle_anchor, cancel_at, cancel_at_period_end, canceled_at, created, current_period_end, current_period_start, customer, ended_at, quantity, start_date, status = value
        #insert subscriber record
        subscriber_data = (subscription_id, billing_cycle_anchor, cancel_at, cancel_at_period_end, canceled_at, created, current_period_end, current_period_start, customer, ended_at,quantity, start_date, status)
        cur.execute(insert_table, subscriber_data)
    
    conn.commit()


In [ ]:
insert_values(conn, df)

In [ ]:
test_df_4_me = sqlio.read_sql_query("SELECT * FROM subscriptions", conn)
test_df_4_me

In [ ]:
conn.close()